# 自然言語処理

ディープラーニング セクションの学習ステップ 8/8。
自然言語処理は、損失が下がる理由を『式』と『数値変化』の両方で確認する構成です。

このステップの到達目標: 損失、勾配、最適化、正則化のつながりをコードで確かめ、モデルの挙動を説明できる状態にします。
前提: ベクトル・行列の計算と、微分の意味を言葉で説明できると理解しやすくなります。

今回の中心語: 「順伝播」、「逆伝播」、「勾配」、「損失」、「正則化」、「自然言語処理」
前ステップ「Transformer」では 自己注意の最小例 → 損失を計算する を確認しました。
ここまでに登場した語: 「順伝播」、「逆伝播」、「勾配」、「損失」、「正則化」、「ニューラルネットワーク」、「損失関数と勾配降下法」、「最適化と正則化」、「畳み込みとCNN」、「画像認識」
セクション全体のゴール: 損失、勾配、最適化、正則化のつながりをコードで確かめ、モデルの挙動を説明できる状態にします。

## 実験 1: トークン埋め込みの最小計算

NLPの入口として、語IDから埋め込みを引いて合成ベクトルを作ります。

In [ ]:
import math
embedding = {0: [0.1, 0.2], 1: [0.4, -0.1], 2: [0.3, 0.5]}
token_ids = [0, 2, 1]
vec = [sum(embedding[t][d] for t in token_ids) / len(token_ids) for d in range(2)]
x = [vec[0], vec[1], 1.0]
w = [0.7, -0.2, 0.05]
z = sum(xi * wi for xi, wi in zip(x, w))
y = 1 / (1 + math.exp(-z))
print('task = nlp-deep-learning', 'vec=', [round(v, 4) for v in vec], 'pred=', round(y, 6))

埋め込みから予測までの流れを、まず小さく確認します。

この節では、順伝播 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 実験 2: 損失を計算する

次に、予測値に対する損失を計算します。学習は損失を下げる方向に進むので、損失の意味を言葉で説明できることが重要です。

In [ ]:
target = 1.0
eps = 1e-9
loss = -(target * math.log(y + eps) + (1 - target) * math.log(1 - y + eps))
print('loss =', round(loss, 6))
print('error =', round(target - y, 6))

損失は『どれだけ外したか』を測る物差しです。物差しがない状態では、改善の方向を決められません。

この節では、順伝播 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 式と実装の往復

1. $z = W x + b$
2. $\theta \leftarrow \theta - \eta \, \nabla_{\theta} L$

## 実験 3: 勾配降下法の一歩

ここで、重みを一回更新する最小実験を行います。更新前後の損失を比較して、学習の方向が正しいかを確認します。

In [ ]:
lr = 0.1
grad = -(target - y) * y * (1 - y)
w_new = [wi - lr * grad * xi for wi, xi in zip(w, x)]
z_new = sum(xi * wi for xi, wi in zip(x, w_new)) + 0.1
y_new = 1 / (1 + math.exp(-z_new))
print('y before/after =', round(y, 6), round(y_new, 6))

この更新で損失が下がれば、勾配方向が合理的だったと言えます。下がらないなら学習率や符号を疑います。

この節では、順伝播 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 実験 4: 正則化の感覚を作る

次に、重みが大きくなりすぎることを抑える正則化項を追加します。過学習対策をコードで体験するのが狙いです。

In [ ]:
l2 = 0.01
weight_norm = sum(wi * wi for wi in w_new)
loss_reg = loss + l2 * weight_norm
print('weight_norm =', round(weight_norm, 6))
print('regularized loss =', round(loss_reg, 6))

正則化は精度を上げる魔法ではなく、汎化の崩れを防ぐ保険です。評価データと合わせて効果を判断してください。

この節では、順伝播 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 実験 5: ミニバッチの考え方

最後に、複数サンプルをまとめて扱う発想を確認します。実務では1件ずつよりバッチ処理が主流です。

In [ ]:
batch = [[0.8, -0.4, 0.2], [0.2, 0.1, -0.3], [0.5, -0.2, 0.7]]
targets = [1.0, 0.0, 1.0]
preds = []
for bx in batch:
    z_b = sum(xi * wi for xi, wi in zip(bx, w_new)) + 0.1
    preds.append(1 / (1 + math.exp(-z_b)))
print('preds =', [round(p, 4) for p in preds])

ミニバッチを扱えるようになると、計算効率と学習安定性の両面で設計の幅が広がります。

この節では、順伝播 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 振り返り

今回のノートで押さえておくべき誤解しやすい点を整理します。

1. 学習率が大きすぎて発散する
2. 検証損失の監視をせず過学習を見逃す
3. 前処理と活性化の相性を無視する

自然言語処理 はこのセクションの最終ステップです。先頭ノートから順に再実行し、流れ全体を確認してください。